In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
# pprevent annoying tensorflow warning
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import warnings
warnings.simplefilter("ignore")
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

if you're problem affected badly by huge numbers then it makes sense that MSE will make the model stay away from those big numbers

https://www.kaggle.com/discussions/questions-and-answers/160699#896638

In [4]:
TRAIN_PATH='../input/vietnamese-foods/Images/Train'
TEST_PATH='../input/vietnamese-foods/Images/Test'
IMAGE_SIZE=(224,224)
BATCH_SIZE=128 #128
VALIDATE_PATH='../input/vietnamese-foods/Images/Validate'
class_count=30

In [5]:
train_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40, 
    width_shift_range = 0.2, 
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True)

validate_generator = ImageDataGenerator(rescale=1./255)
test_generator = ImageDataGenerator(rescale=1./255)

train_data = train_generator.flow_from_directory(TRAIN_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
validate_data = validate_generator.flow_from_directory(VALIDATE_PATH, target_size=IMAGE_SIZE, batch_size=BATCH_SIZE)
test_data = test_generator.flow_from_directory(TEST_PATH, target_size=IMAGE_SIZE)

In [6]:
classes=list(train_data.class_indices.keys())
print(f'classes:{classes} \r')
class_indices=list(train_data.class_indices.values())
print(f'class_indices:{class_indices}')
labels=test_data.labels
print(f'label :{labels}')

In [ ]:
train_data

In [2]:
pretrained_model = VGG19(weights='imagenet', include_top=False)
last_output = pretrained_model.output
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(30, activation='softmax')(x)
model1 = Model(inputs=pretrained_model.input, outputs=outputs)
model1.summary() 

In [ ]:
# base_model=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 

In [ ]:
# create an efficientNetB0 model to use for transfer learning
# img_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)#(224, 224, 3)
# model_name='EfficientNetB0'
# base_model=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 
# # Note you are always told NOT to make the base model trainable initially- that is WRONG you get better results leaving it trainable
# # base_model.trainable=True
# x=base_model.output
# x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
# x = Dense(1024, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dropout(rate=.5, seed=123)(x)
# x = Dense(512, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
#                 bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x=Dropout(rate=.45, seed=123)(x)         
# output=Dense(class_count, activation='softmax')(x)
# model=Model(inputs=base_model.input, outputs=output)
lr=.001 # start with this learning rate
# model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy']) 

In [8]:
pretrained_model = VGG19(weights='imagenet', include_top=False)
last_output = pretrained_model.output
x = GlobalAveragePooling2D()(last_output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.4)(x)
outputs = Dense(30, activation='softmax')(x)
model = Model(inputs=pretrained_model.input, outputs=outputs)

In [9]:
for layer in pretrained_model.layers: layer.trainable = False #,
#this data contain ,multiclass so i used 'categorical_crossentropy'
model.compile(optimizer='rmsprop',loss='mse', metrics=['accuracy']) 
#categorical_crossentropy

In [10]:
# create 2 useful callbacks, one to control the learning rate, and one to control early stopping based on validation loss
rlronp=tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2,verbose=1)
# estop=tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=4, verbose=1,restore_best_weights=True)
# #rlronp,
# callbacks=[estop]

In [ ]:
PATH = 'Models/VGG19'
BASE_MODEL_BEST = os.path.join(PATH, 'base_model_best.hdf5')

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
# base_checkpointer = ModelCheckpoint(
#     filepath = BASE_MODEL_BEST, 
#     save_best_only = True, 
#     verbose = 1
# )

# fine_tune_checkpointer = ModelCheckpoint(
#     filepath = FINE_TUNE_MODEL_BEST, 
#     save_best_only = True,
#     verbose = 1, )


# Stop if no improvement after 3 epochs


**I choosed RMSprop (with default values), it is a very effective optimizer. The RMSProp update adjusts the Adagrad method in a very simple way in an attempt to reduce its aggressive, monotonically decreasing learning rate. We could also have used Stochastic Gradient Descent ('sgd') optimizer, but it is slower than RMSprop.**

https://www.kaggle.com/code/yassineghouzam/introduction-to-cnn-keras-0-997-top-6#4.-Evaluate-the-model

**Adaptive Gradient Algorithm (AdaGrad)** 
    
that maintains a per-parameter learning rate that improves performance on problems with sparse gradients (e.g. natural language and computer vision problems)          
.
**Root Mean Square Propagation (RMSProp)**
      
that also maintains per-parameter learning rates that are adapted based on the average of recent magnitudes of the gradients for the weight (e.g. how quickly it is changing). This means the algorithm does well on online and non-stationary problems (e.g. noisy)

https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

In [12]:
# train the model
epochs=20
history=model.fit(x=train_data ,epochs=epochs, verbose=1,validation_data=validate_data,
                  callbacks = [rlronp, early_stopping],shuffle=False, initial_epoch=0)

In [ ]:
# vẽ lại quá trình học
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','Validation'])
plt.show()

In [ ]:
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import matplotlib.pyplot as plt
filename = "../input/fruitdataset/train/tomato/Image_11.jpg"

img = load_img(filename,target_size=(224,224))
img_show = plt.imshow(img)
plt.show()
img = img_to_array(img)
img = img.reshape(1,224,224,3)
img = img.astype('float32')
img = img/255
kq=np.argmax(model.predict(img),axis=-1)
if(kq==0):
    print("Banh beo")
if(kq==1):
    print("Bot loc")
if(kq==2):
    print("Banh Can")
if(kq==3):
    print("Banh canh")
if(kq==4):
    print("Banh chung")
if(kq==5):
    print("Banh cuon")
if(kq==6):
    print("Banh duc")
if(kq==7):
    print("Banh gio")
if(kq==8):
    print("Banh khot")
if(kq==9):
    print("Banh mi")
if(kq==10):
    print("Banh pia")
